<a href="https://colab.research.google.com/github/vsky0/Gemini-API-General/blob/master/Gemini_Exercise_Apollo_11_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation And Setup

Installing the required modules and importing.

In [1]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytessaract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

In [1]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

Importing Modules

In [2]:
import time
from tqdm import tqdm

import pathlib

import google.generativeai as genai

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

import pandas as pd
from PIL import Image

import pytesseract

from IPython.display import Markdown

Load and Configure the API KEY

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('G_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

Selecting the model, gemini-1.5-flash

In [ ]:
model_name = 'gemini-1.5-flash'
model = genai.GenerativeModel(model_name)

# Gemini Final Excercise


## Data Preparation
Getting the resources from the link

In [4]:
!wget -O resources.zip "https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip"

--2024-07-20 05:06:12--  https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip
Resolving video.udacity-data.com (video.udacity-data.com)... 104.19.140.72, 104.19.139.72, 104.19.142.72, ...
Connecting to video.udacity-data.com (video.udacity-data.com)|104.19.140.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286142532 (273M) [application/zip]
Saving to: ‘resources.zip’

resources.zip       100%[===================>] 272.89M   152MB/s    in 1.8s    

2024-07-20 05:06:14 (152 MB/s) - ‘resources.zip’ saved [286142532/286142532]



uzip the resources

In [5]:
!unzip resources.zip

Archive:  resources.zip
   creating: resources/
  inflating: __MACOSX/._resources    
   creating: resources/video/
  inflating: __MACOSX/resources/._video  
  inflating: resources/.DS_Store     
  inflating: __MACOSX/resources/._.DS_Store  
   creating: resources/audio/
  inflating: __MACOSX/resources/._audio  
   creating: resources/text/
  inflating: __MACOSX/resources/._text  
  inflating: resources/video/Apollo11PlaqueComparison.mov  
  inflating: __MACOSX/resources/video/._Apollo11PlaqueComparison.mov  
  inflating: resources/video/Apollo11Intro.mov  
  inflating: __MACOSX/resources/video/._Apollo11Intro.mov  
  inflating: resources/video/Apollo11MoonwalkMontage.mov  
  inflating: __MACOSX/resources/video/._Apollo11MoonwalkMontage.mov  
  inflating: resources/video/OneSmallStepCompilation.mov  
  inflating: __MACOSX/resources/video/._OneSmallStepCompilation.mov  
  inflating: resources/video/RaisingTheAmericanFlag.mov  
  inflating: __MACOSX/resources/video/._RaisingTheAmericanFl

In [6]:
data_dir = pathlib.Path("resources/")
all_file_names = [str(file) for file in data_dir.rglob("*") if file.is_file() and not file.name.startswith('.')]

In [7]:
for file_name in all_file_names:
    print(file_name)

print(len(all_file_names))

resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/video/Apollo11Intro.mov
resources/video/Apollo11PlaqueComparison.mov
resources/video/OneSmallStepCompilation.mov
resources/video/BuzzDescendsCompilation.mov
resources/video/Apollo11MoonwalkMontage.mov
resources/video/RaisingTheAmericanFlag.mov
resources/text/images-020.jpg
resources/text/images-023.jpg
resources/text/images-333.jpg
14


## Retrieval Augmented Generation - RAG

General Steps that involves a RAG are

1. Data Preparation
2. Datat Extraction and Summarization
3. Embedding Generation
4. Creating a Vector Database
5. Querying the RAG System

### Case: Text

Using the Tesseraccct OCR to extract text from images of the NASA report.

In [8]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

Funcction extract the text from the images and summerazie them.

In [10]:
def create_text_summary():
  path = pathlib.Path("resources/text")

  text_summary_prompt = f"""You are an assistant tailored for summarizing text for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw text.
  Give a concise summary of the text that is well optimized for retrieval. Here is the text."""

  images = []
  text_summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue

    image = Image.open(f)
    response = model.generate_content(
                [text_summary_prompt, pytesseract.image_to_string(image)]
              )

    images.append(image)
    text_summaries.append(response.text)

  return images, text_summaries

altering the safety settings

In [12]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]
model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)


calling the function create_text_summary

In [13]:
image_files, text_summaries = create_text_summary()

checking the generated text summaries.

In [14]:
for text_summary in text_summaries:
  print(text_summary)


This document outlines the Apollo 11 mission plan, including the schedule of operations and crew activities. It was prepared by the Flight Planning Branch with support from TRW Systems. The plan aims to fulfill the test objectives defined in the Mission Requirements for a G Type Mission Lunar Landing.  Trajectory parameters used were for a July 16, 1969 launch. Changes to this document must be submitted through a Crew Procedures Change Request to the Crew Procedures Control Board (CPCB). 

This document outlines the Apollo 11 mission timeline from launch to lunar orbit insertion (LOI). It includes details on launch time, Earth orbit insertion, Translunar Coast events such as transposition, docking, and midcourse corrections. Key events like SIVB separation and propulsive venting are also mentioned. 

This document outlines ground rules and assumptions for EPS analysis, covering battery usage, equipment operation, and system configurations for a lunar mission. Assumptions include a 30-m

creating a chroma database using the generatd summaries.

In [17]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    title = 'Custome Query'
    return genai.embed_content(model = model,
                               content = input,
                               task_type = "retrieval_document",
                               title = title)["embedding"]


In [18]:
def create_chroma_db(documents, name):
    chroma_client = chromadb.Client()
    db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

    for i, d in enumerate(documents):
      db.add(
          documents = d,
          ids = str(i)
      )
    return db

In [19]:
text_db = create_chroma_db(text_summaries, "text_nasa")

In [22]:
data = {
    'embeddings': text_db.peek()['embeddings'],
    'documents': text_db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.06969980895519257, 0.01844603940844536, 0.0...",This document outlines the Apollo 11 mission p...
1,"[0.05890379101037979, -0.015314590185880661, 0...",This document outlines the Apollo 11 mission t...
2,"[0.0705656185746193, -0.0014043324626982212, -...",This document outlines ground rules and assump...


function to fetch the relevant files.

In [23]:
def get_relevant_files(query, db):
  results = db.query(query_texts = [query], n_results = 3)
  return results["ids"][0]

In [24]:
files = get_relevant_files("Apollo 11 Fight Plan", text_db)
print(files)

['0', '1', '2']


### Case: Video and Audio

In [28]:
def create_summary(modality):
  path = data_dir / modality

  summary_prompt = f"""You are an assistant tailored for summarizing {modality} for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw {modality}.
  Give a concise summary of the {modality} that is well optimized for retrieval. Here is the {modality}."""

  files = []
  summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue
    print(f)

    if modality == "text":
      file = Image.open(f)
      response = model.generate_content([summary_prompt, pytesseract.image_to_string(file)])

    else:
      file = genai.upload_file(f)

      while file.state.name == "PROCESSING":
        print("Waiting for video file upload...\n", end='')
        time.sleep(5)
        file = genai.get_file(file.name)

      response = model.generate_content([summary_prompt, file])

    files.append(file)
    summaries.append(response.text)

  return files, summaries

creating directory for all of the modalities

In [30]:
all_files = []
all_summaries = []

for modality_type in ["audio", "text", "video"]:
  files, summaries = create_summary(modality_type)
  all_files.extend(files)
  all_summaries.extend(summaries)

resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/text/images-020.jpg
resources/text/images-023.jpg
resources/text/images-333.jpg
resources/video/Apollo11Intro.mov
Waiting for video file upload...
resources/video/Apollo11PlaqueComparison.mov
Waiting for video file upload...
resources/video/OneSmallStepCompilation.mov
Waiting for video file upload...
resources/video/BuzzDescendsCompilation.mov
Waiting for video file upload...
resources/video/Apollo11MoonwalkMontage.mov
Waiting for video file upload...
resources/video/RaisingTheAmericanFlag.mov
Waiting for video file upload...


In [31]:
db = create_chroma_db(all_summaries, "nasa")

In [32]:
data = {
    'embeddings': db.peek()['embeddings'],
    'documents': db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.0513673759996891, -0.020518522709608078, -0...",A group of astronauts are discussing the readi...
1,"[-0.010250307619571686, 0.010288766585290432, ...",Two people are talking about flying. One perso...
2,"[0.013874443247914314, -0.037314269691705704, ...",The Apollo 11 EVA Television Comparison 1 vide...
3,"[0.043346717953681946, -0.04161280766129494, -...",A side-by-side comparison of Buzz Aldrin desce...
4,"[0.04568318650126457, 0.021823478862643242, -0...",Apollo 11 astronauts Neil Armstrong and Buzz A...
5,"[0.042539648711681366, -0.0051874322816729546,...",Side-by-side comparison of NASA archival foota...
6,"[0.04963104426860809, -0.017791161313652992, -...",A group of astronauts are discussing taking pi...
7,"[0.02152538113296032, 0.031136520206928253, -0...","A conversation about plane activity and speed,..."
8,"[-0.02030533365905285, 0.02446693740785122, -0...",Two people are discussing flight plans and the...
9,"[0.06837735325098038, 0.0098823681473732, 0.00...",This document outlines the Apollo 11 flight pl...


In [33]:
files = get_relevant_files("Communication with Mission Control",db)
print(files)

['0', '2', '5']


function to query the RAG system

In [34]:
def query_rag(query, db):
  files = get_relevant_files(query, db)
  prompt = [all_files[int(f)] for f in files]
  prompt.append("Generate a response to the query using the provided files. Here's the query.")
  prompt.append(query)
  return model.generate_content(prompt).text, [all_file_names[int(f)] for f in files]

Querying the RAG system.

In [35]:
for response in query_rag("Explain what happened with Apollo 11 Mission.", db):
  print(response)

The Apollo 11 mission was the first crewed mission to land on the moon. The mission launched on July 16, 1969, and landed on the moon on July 20, 1969.  Neil Armstrong was the first person to walk on the moon. He famously said "That's one small step for man, one giant leap for mankind". 
The crew of Apollo 11 consisted of Neil Armstrong, Buzz Aldrin and Michael Collins.
The mission lasted 8 days, 3 hours, and 18 minutes. 
Apollo 11 mission was a huge success for the US space program and it was a major victory in the space race between the United States and the Soviet Union. 
This mission marked a significant milestone in human history and inspired generations of space explorers.

['resources/video/BuzzDescendsCompilation.mov', 'resources/video/Apollo11PlaqueComparison.mov', 'resources/text/images-023.jpg']


In [36]:
for response in query_rag("What happens at the Translunar Coast in the Missiion Description?", db):
  print(response)

After the TLI (Translunar Injection), the spacecraft is in a free lunar return trajectory. The following major events occur prior to LOI: transposition, docking and LM ejection, including SIVB photography, separation from SIVB and a CSM evasive maneuver, SIVB propulsive venting of propellants (slingshot), two series of P23 cislunar navigation sightings, star/earth horizon, consisting of five sets at 06:00 GET and five sets at 24:30 GET, four midcourse corrections which take place at TLI + 9, TLI + 24, LOI - 22 and LOI - 5 hours with ΔV nominally zero. 
['resources/video/Apollo11PlaqueComparison.mov', 'resources/video/Apollo11Intro.mov', 'resources/text/images-023.jpg']


In [ ]:
import textwrap

In [39]:
# function to markdown the response of gemini
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [41]:
# ask more questions about Apollo 11
for response in query_rag("How do you prove that Apollo 11 mission was not fake, provide related info to prove me wrong.",db):
  print(to_markdown(response))

ReadTimeout: HTTPConnectionPool(host='localhost', port=43931): Read timed out. (read timeout=60.0)